## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 아산시청 위도, 경도
Asan = [36.789882248764656, 127.00274491353838]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

#### 아산시 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2405_001/_census_data_2023_bnd_dong_bnd_dong_34040_2023_2023/bnd_dong_34040_2023_2023_2Q.shp"
asan_gdf = gpd.read_file(shapefile_path)
asan_gdf = asan_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/12.아산시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 아산시 실폭도로

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/7.아산시_실폭도로.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
road_df = pd.json_normalize(geojson_data['features'])
road_df['geometry'] = road_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# road_df 데이터프레임을 GeoDataFrame으로 변환
road_df = gpd.GeoDataFrame(road_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=14)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
roal_geojson = road_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)
# 맵 출력
m

#### 아산시 교차로

In [44]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/9.아산시_교차로.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
crossroad_df = pd.json_normalize(geojson_data['features'])
crossroad_df['geometry'] = crossroad_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crossroad_df 데이터프레임을 GeoDataFrame으로 변환
crossroad_df = gpd.GeoDataFrame(crossroad_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=14)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
roal_geojson = crossroad_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="교차로",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)
# 맵 출력
m

#### 아산시 표준노드정보

In [66]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/14.아산시_표준노드정보.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
node_df = pd.json_normalize(geojson_data['features'])
node_df['geometry'] = node_df['geometry.coordinates'].apply(lambda x : make_point(x))
# node_df 데이터프레임을 GeoDataFrame으로 변환
node_df = gpd.GeoDataFrame(node_df, geometry='geometry')

In [ ]:
# map 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=14)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)

# 노드 위치 매핑 및 find_radius_from_area 함수로 노드 면적만큼 radius 설정
m_market = folium.FeatureGroup(name="노드", overlay=True)
for _, row in node_df.iterrows() :
    popup_text = f"{row['properties.NODE_TYPE']} - {row['properties.NODE_NAME']}"
    folium.Circle(location=(row["geometry"].y, row["geometry"].x), radius=3, color='red', weight = 1,
            fill='red', name="표준노드").add_to(m_market).add_child(folium.Popup(popup_text, max_width=200))
m_market.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

#### 아산시 표준링크정보

In [75]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/15.아산시_표준링크정보.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
link_df = pd.json_normalize(geojson_data['features'])
link_df['geometry'] = link_df['geometry.coordinates'].apply(lambda x : make_lin(x))
# link_df 데이터프레임을 GeoDataFrame으로 변환
link_df = gpd.GeoDataFrame(link_df, geometry='geometry')

#### 아산시 상세 도로망 & 도로명주소 데이터 필터링 및 통합
##### 상세도로망으로도 충분히 분석 가능하기 때문에, 도로명주소 데이터는 굳이 안써도 되겠다 판단하였음

In [9]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/4.아산시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

# GeoJSON 파일 불러오기 
with open('SBJ_2405_001/5.아산시_도로명주소(도로).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_road_df = pd.json_normalize(geojson_data['features'])
roadname_road_df['geometry'] = roadname_road_df['geometry.coordinates'].apply(lambda x: make_lin(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadname_road_df = gpd.GeoDataFrame(roadname_road_df, geometry='geometry')

# 통합도로망 df
tot_roadsystem = pd.concat([roadsystem_df[['geometry']], roadname_road_df[['geometry']]])

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
roal_geojson = roadsystem_df['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)
# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)
    
# 맵 출력
m

#### 아산시 추정교통량

In [12]:
# 도로 id와 geometry를 매핑한 딕셔너리 생성 : 추정교통량 data와 매핑 예정
road_geo_dic = dict(zip(roadsystem_df['properties.link_id'], roadsystem_df['geometry']))

traffic_df = pd.read_csv('SBJ_2405_001/3.아산시_추정교통량.csv')
traffic_df['year'] = traffic_df['year'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].apply(lambda x : x[:-2])
traffic_df

C:\Users\HJK\AppData\Local\Temp\ipykernel_27976\2271375159.py:4: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



,year,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT
0,2020,561636912,107,0.097,NaN,충청남도,아산시,온양3동,2,27,25,0,2
1,2020,561636912,107,0.097,NaN,충청남도,아산시,온양3동,2,27,25,0,2
2,2020,561636915,107,0.100,NaN,충청남도,아산시,온양3동,2,30,28,0,2
3,2020,561622068,107,0.105,NaN,충청남도,아산시,온양3동,2,26,24,0,2
4,2020,561622068,107,0.105,NaN,충청남도,아산시,온양3동,2,26,24,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726554,2022,561624294,107,0.149,어의정로,충청남도,아산시,온양5동,10,364,329,6,29
726555,2022,561624294,107,0.149,어의정로,충청남도,아산시,온양5동,10,263,234,5,24
726556,2022,561624335,107,0.029,삼성로,충청남도,아산시,탕정면,10,428,275,19,135
726557,2022,561624335,107,0.029,삼성로,충청남도,아산시,탕정면,10,516,332,23,161


In [13]:
# 딕셔너리 생성 및 year & DataFrame 매핑
traffic_df_year_dic = {}
year_list = ['2020', '2021', '2022']

for year in year_list:
    traffic_df_year = traffic_df[traffic_df['year'] == year].copy() # 특정 연도로만 필터링
    traffic_df_year['geometry'] = traffic_df_year['link_id'].map(road_geo_dic)
    # 일별 누적교통량만을 사용하기 위해 timeslot == all인 row만 필터링
    traffic_df_year = gpd.GeoDataFrame(traffic_df_year, geometry='geometry')
    traffic_df_year = traffic_df_year[traffic_df_year['timeslot'] == 'all']
    concat_df = pd.DataFrame()
    for road_id in tqdm(traffic_df_year['link_id'].unique()):
        subset = traffic_df_year[traffic_df_year['link_id'] == road_id]
        column_sums = subset.iloc[:,9:13].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        subset_front = pd.DataFrame(subset.iloc[0,:9]).transpose().reset_index(drop=True)
        subset_back = pd.DataFrame(subset.iloc[0,13:]).transpose().reset_index(drop=True)
        re_subset = pd.concat([subset_front, column_sums_df, subset_back], axis=1)
        concat_df = pd.concat([concat_df, re_subset])
    else:
        traffic_df_year_dic[year] = concat_df

100%|██████████| 5911/5911 [00:31<00:00, 184.78it/s]


#### 연도별 도로망 - 교통량 시각화

In [ ]:
#  교통량에 따라 도로망 색상을 지정하는 함수
def pick_color(traffic, dataframe):
    desc_lst = subset.describe().iloc[4:7,0].tolist()
    top_25 = desc_lst[-1]
    top_50 = desc_lst[-2]
    top_75 = desc_lst[-3]
    if traffic > top_25:
        return 'red'
    elif top_25 > traffic >= top_50:
        return 'orange'
    elif top_50 > traffic >= top_75:
        return 'yellow'
    elif top_75 > traffic >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 1);
     "> &nbsp; 교통량 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 0% ~ 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 25% ~ 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 59% ~ 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 75% ~ 100%
     </div>
     """

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = roadsystem_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = [str(year) for year in range(2020, 2023)]
layers_lst = [m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2020부터 2022까지의 연도
    subset = traffic_df_year_dic[year].copy()
    subset.dropna(subset=['geometry'], inplace=True)
    # 'road'(LINESTRING)을 기준으로 그룹화
    for road in tqdm(subset['geometry'].unique()):
        subset_road = subset[subset['geometry'] == road].copy()
        traffic = subset_road['ALL_AADT'].iloc[0]
        color = pick_color(traffic, subset)
        popup_text = f"<div>{subset_road['road_nm']} - {subset_road['ALL_AADT']}</div>"
        folium.GeoJson(
            subset_road['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 8}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text, max_width=500), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [11]:
m.save("visualization/2. 아산시 교통분석/아산시 교통량 시각화.html")

#### 연도별 교통량 HEATMAP

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = roadsystem_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)
m_2022 = folium.FeatureGroup(name="2022년", overlay=False)

year_lst = [str(year) for year in range(2020, 2023)]
layers_lst = [m_2020, m_2021, m_2022]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2020부터 2022까지의 연도
    subset = traffic_df_year_dic[year]
    subset.dropna(subset=['geometry'], inplace=True)
    # 'ALL_AADT' 열의 값을 교통량으로 사용하고 'geometry' 열의 값을 위치 정보로 사용하여 HeatMap 생성
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], traffic] for point, traffic in tqdm(zip(subset['geometry'], subset['ALL_AADT']))]
    HeatMap(heat_data, max_opacity=0.8, radius=15).add_to(layers_dic[year])

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [27]:
m.save("visualization/2. 아산시 교통분석/아산시 교통량 HEATMAP.html")

### 교통관련시설 및 교통사고이력 시각화

#### 아산시 교통단속카메라 현황

In [ ]:
traffic_cam_df = pd.read_csv('SBJ_2405_001/10.아산시_무인교통단속카메라현황.csv')
traffic_cam_df = geo_transform(traffic_cam_df)

#### 아산시 신호등 현황 

In [ ]:
# 신호등 데이터프레임을 GeoPandas 데이터프레임으로 변환
traffic_lights_df = pd.read_csv('SBJ_2405_001/18.아산시_신호등현황.csv')
traffic_lights_df = geo_transform(traffic_lights_df)

#### 아산시 과속방지턱정보

In [ ]:
# 과속방지턱 데이터프레임을 GeoPandas 데이터프레임으로 변환
speed_bump_df = pd.read_csv('SBJ_2405_001/19.아산시_과속방지턱정보.csv')
speed_bump_df = geo_transform(speed_bump_df)

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = roadsystem_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 과속방지턱 위치 mapping
m_speed_bump = folium.FeatureGroup(name="과속방지턱", overlay=True)
for _, row in speed_bump_df.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=3, color='yellow', weight = 1,
            fill='red', name="과속방지턱").add_to(m_speed_bump)
m_speed_bump.add_to(m)

# 신호등 위치 mapping
m_traffic_lights = folium.FeatureGroup(name="신호등", overlay=True)
for _, row in traffic_lights_df.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=3, color='red', weight = 1,
            fill='red', name="신호등").add_to(m_traffic_lights)
m_traffic_lights.add_to(m)

# 교통단속카메라 위치 매핑
m_traffic_cam = folium.FeatureGroup(name="교통단속카메라", overlay=True)
for _, row in traffic_cam_df.iterrows() :
    folium.Circle(location=(row['lat'], row['lon']), radius=3, color='black', weight = 1,
            fill='red', name="교통단속카메라").add_to(m_traffic_cam)
m_traffic_cam.add_to(m)

# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [86]:
m.save("visualization/2. 아산시 교통분석/아산시 교통관련시설위치 시각화.html")

#### 아산시 교통사고 이력

In [59]:
# GeoJSON 파일 불러오기
with open('SBJ_2405_001/6.아산시_교통사고이력.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
accident_df = pd.json_normalize(geojson_data['features'])
accident_df['geometry'] = accident_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# accident_df 데이터프레임을 GeoDataFrame으로 변환
accident_df = gpd.GeoDataFrame(accident_df, geometry='geometry')
accident_df.columns = [i.split("_")[-1] for i in accident_df.columns]
# 교통사고 건수 통합(3개년 통합)
accident_df['tot_acc'] = accident_df['2020'] + accident_df['2021'] + accident_df['2022']
accident_df['tot_acc'] = accident_df['tot_acc'].fillna(0)
accident_df =  accident_df[accident_df['tot_acc'] != 0]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def pick_color(value):
    colors = [
        (0/255, 128/255, 0/255, 1),  # Green: RGBA(0, 128, 1)
        (255/255, 255/255, 0/255, 1),  # Yellow: RGBA(255, 255, 1)
        (255/255, 165/255, 0/255, 1),  # Orange: RGBA(255, 165, 1)
        (255/255, 0/255, 0/255, 1)   # Red: RGBA(255, 0, 1)
    ]
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', colors, N=33)
    normalized_value = (value - 1) / 32.0
    color = cmap(normalized_value)
    color_str = f"RGBA({int(color[0] * 255)}, {int(color[1] * 255)}, {int(color[2] * 255)}, {int(color[3])})"
    return color_str

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 1);
     "> &nbsp; 사고건수 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 0% ~ 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 25% ~ 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 59% ~ 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 75% ~ 100%
     </div>
     """

# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = roadsystem_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
for idx, row in tqdm(accident_df.iterrows()):
    traffic = row['tot_acc']
    color = pick_color(traffic)
    popup_text = f"<div>{row['tot_acc']}건</div>"
    folium.GeoJson(
        row['geometry'].__geo_interface__,
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': 'black',
            'weight': 1}
    ).add_to(m).add_child(folium.Popup(popup_text, max_width=500), name=str(year))


# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

In [71]:
m.save("visualization/2. 아산시 교통분석/아산시 교통사고이력 시각화.html")

#### 아산시 교통사고 HEATMAP

In [ ]:
# 지도 생성
m = folium.Map(location=Asan, tiles='cartodbpositron', zoom_start=11)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('cartodbpositron', overlay=False).add_to(m)
grid_geojson = roadsystem_df['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)
# 행정구역 polygon 추가
for _, row in asan_gdf.iterrows() :
    folium.GeoJson(
        row['geometry'],
        name=row['ADM_NM'],
        style_function=lambda feature: {
            'fillColor': 'none',
            'color': 'black',
            'weight': 1
        }).add_to(m)

# 'ALL_AADT' 열의 값을 교통량으로 사용하고 'geometry' 열의 값을 위치 정보로 사용하여 HeatMap 생성
heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], acc] for point, acc in tqdm(zip(accident_df['geometry'], accident_df['tot_acc']))]
HeatMap(heat_data, max_opacity=0.8, radius=15).add_to(m)

# 지도 출력
m

In [80]:
m.save("visualization/2. 아산시 교통분석/아산시 교통사고이력 HEATMAP.html")